# Alachua County restaurant inspection analysis
This will take restaurant inspection data by the state of Florida and format it in a more reader-friendly way for publication in print and online. We'll filter for the most egregious current violations at restaurants in Alachua County.

__After importing Pandas and Datetime, this reads in the state summary report year-to-date for District 5, which includes Alachua County, and adds an exception in case the file is not found (output probably needs to be set as variable so it can be written into the output file). The raw file has no headers and 82 columns. So this removes all but five columns and adds headers for those. Finally, it displays the first five rows of values.__

In [1]:
import pandas as pd
import bs4
import datetime 
import re
import lxml
from urllib.request import urlopen
from urllib.error import HTTPError

In [2]:
try:
    insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                               usecols=[2,14,18,80,81])
    
except IOError:
    print("The file is not accessible.")
insp.columns = ["CountyName", "InspectDate", "NumHighVio", "LicenseID", "VisitID"]

__This creates a DataFrame named 'alachua', filters out records for Alachua County:__

In [3]:
# filter for alachua county restaurants with high violations
insp[(insp.CountyName == 'Alachua') & (insp.NumHighVio > 0)]

,CountyName,InspectDate,NumHighVio,LicenseID,VisitID
0,Alachua,07/12/2018,2.0,2129032,6580341
2,Alachua,07/02/2018,1.0,6605529,6632858
3,Alachua,07/13/2018,5.0,2129251,6674765
5,Alachua,07/03/2018,7.0,4070634,6602877
8,Alachua,07/05/2018,1.0,5768844,6618656
9,Alachua,07/03/2018,2.0,4332605,6606469
11,Alachua,07/13/2018,2.0,2129489,6579450
12,Alachua,07/09/2018,2.0,4352477,6606018
13,Alachua,07/09/2018,2.0,4182724,6604435
14,Alachua,07/05/2018,2.0,2129510,6671597


In [4]:
# change date string to date object, sort most recent# chang 
insp['InspectDate'] = pd.to_datetime(insp['InspectDate'])
insp = insp.sort_values('InspectDate', ascending=False)

In [5]:
# prefer to let user set timedelta
today = pd.to_datetime('today')
startDay = datetime.date.today() - datetime.timedelta(days=14)
insp = insp[(insp['InspectDate'] > startDay) & (insp['InspectDate'] < today)]
len(insp)

442

In [6]:
# takes LicenseID and VisitID, passes it into the urls for detailed reports# takes 
result = []
for index, rows in insp.iterrows():
    visitID = rows['VisitID']
    licID = rows['LicenseID']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &licid= %s" % (visitID, licID)
    urls = urls.replace(' ', '')
    result.append(urls)

urlList = result
print(urlList) #not needed later
len(urlList) #not needed later

['https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6602203&licid=4011496', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6611188&licid=5374041', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6675628&licid=5711719', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6674747&licid=6865269', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6674180&licid=7060778', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6600413&licid=3885537', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6676840&licid=5569374', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6576537&licid=2176019', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6675619&licid=3058746', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6677115&licid=2976536', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6653700&licid=6882305'

442

In [7]:
urlList = ['https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6353619&licid=6683171', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6452493&licid=5874202']
# loop through the url list to gather inspection details
detailsLib = {}
def get_inspect_detail(html):
    html = urlopen(urlList)
    soup = bs4.BeautifulSoup(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:]
    result = []
    for detail in details:
        siteName = details[0].text
        licNum = details[2].text
        siteRank = details[4].text
        expDate = details[6].text
        primeStatus = details[8].text
        secStatus = details[10].text
        siteAddress = details[12].text
        inspectResult = details[20].text
        
#       observed1 = details[34].get_text
#       observed2 = details[36].text
#       observed3 = details[38].text
#       observed4 = details[40].text
#       observed5 = details[42].text
#       observed6 = details[44].text
#       observed7 = details[46].text
#       observed8 = details[48].text
#       observed9 = details[50].text
#       observed10 = details[52].text

        detailsLib = {
            'Restaurant': siteName,
            'License': licNum,
            'Rank': siteRank,
            'Expires': expDate,
            'Primary': primeStatus,
            'Secondary': secStatus,
            'Address': siteAddress,
            'Result': inspectResult,
            
#            'Observed1': observed1,
#            'Observed2': observed2,
#            'Observed3': observed3,
#            'Observed4': observed4,
#            'Observed5': observed5,
#            'Observed6': observed6,
#            'Observed7': observed7,
#            'Observed8': observed8,
#            'Observed9': observed9,
#            'Observed10': observed10
        }
        result.append(detailsLib)
        
    return result

#all_results = []
for inspurl in urlList:
    html = urlopen(inspurl)
    #details = get_inspect_detail(url)
    #all_results.extend(details)

In [11]:
detailsLib['License]

NameError: name 'siteName' is not defined

In [ ]:
def clean_up(text, strip_chars=[], replace_extras={}):
    # Handle strip_chars
    strip_items = '|'.join(re.escape(s) for s in strip_chars)
    strip_re = r'^(?:{}|\s)+|(?:{}|\s)+$'.format(strip_items, strip_items)
    text = re.sub(strip_re, '', text, re.MULTILINE)

    # Normalize whitespace and handle replace_extras
    replace_keys = list(replace_extras.keys())
    replace_keys.sort(key=len, reverse=True)
    replace_re = '|'.join([re.escape(s) for s in replace_keys] + [r'\s+'])
    return re.sub(
        replace_re,
        lambda match: replace_extras.get(match.group(), ' '),
        text
    ).strip()

clean_up(str(details))
print(details)

In [ ]:
def getNgrams(content, n):
    content = re.sub('\n|[[\d+\]]', ' ', content)
    content = bytes(content, 'UTF-8')
    content = content.decode('ascii', 'ignore')
    content = content.split(' ')
    content = [word for word in content if word != '']
    output = []
    for i in range(len(content)-n+1):
        output.append(content[i:i+n])
    return ouput

getNgrams(content, 2)

__Time for some Beautiful Soup.__